# Audio Feature Extraction Experiments

This notebook explores different techniques for extracting relevant features from audio files that can be used for tablature generation.

In [2]:
# Import necessary libraries
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import pandas as pd
from IPython.display import Audio

# Set up paths
DATA_DIR = '../data/raw/'
OUTPUT_DIR = '../data/processed/features/'

## Load Sample Audio File

Let's load a sample audio file and examine its properties.

In [3]:
# Load an example audio file
def load_audio(file_path, sr=22050):
    """Load an audio file and return the waveform and sample rate"""
    y, sr = librosa.load(file_path, sr=sr)
    return y, sr

# Placeholder - replace with your actual file
# file_path = os.path.join(DATA_DIR, 'example_guitar.wav')
# y, sr = load_audio(file_path)
# print(f'Audio loaded: {len(y)/sr:.2f} seconds, {sr} Hz sample rate')
# Audio(y, rate=sr)

## Feature Extraction Functions

We'll implement several feature extraction techniques relevant for guitar tab generation.

In [4]:
def extract_pitch(y, sr, fmin=80, fmax=800):
    """Extract pitch information using librosa's pitch tracking"""
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr, fmin=fmin, fmax=fmax)
    return pitches, magnitudes

def extract_onsets(y, sr):
    """Detect note onsets in the audio"""
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    onsets = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)
    return onsets, onset_env

def extract_chromagram(y, sr):
    """Extract chromagram for harmony analysis"""
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
    return chroma

def extract_features(y, sr):
    """Extract all features and return as a dictionary"""
    features = {
        'pitches': extract_pitch(y, sr),
        'onsets': extract_onsets(y, sr),
        'chroma': extract_chromagram(y, sr)
    }
    return features

## Visualization Functions

Functions to visualize the extracted features.

In [5]:
def plot_waveform(y, sr):
    plt.figure(figsize=(12, 4))
    librosa.display.waveshow(y, sr=sr)
    plt.title('Waveform')
    plt.tight_layout()
    plt.show()

def plot_chromagram(chroma, sr):
    plt.figure(figsize=(12, 4))
    librosa.display.specshow(chroma, y_axis='chroma', x_axis='time', sr=sr)
    plt.colorbar()
    plt.title('Chromagram')
    plt.tight_layout()
    plt.show()

def plot_onsets(y, sr, onsets, onset_env):
    plt.figure(figsize=(12, 4))
    times = librosa.times_like(onset_env, sr=sr)
    plt.plot(times, onset_env, label='Onset strength')
    plt.vlines(librosa.frames_to_time(onsets, sr=sr), 0, onset_env.max(), 
              color='r', alpha=0.7, linestyle='--', label='Onsets')
    plt.legend()
    plt.title('Onset Detection')
    plt.tight_layout()
    plt.show()

## Feature Extraction Pipeline

Now let's put everything together into a pipeline that can be applied to multiple audio files.

In [6]:
def process_audio_file(file_path, output_dir=None):
    """Process a single audio file and extract features"""
    print(f'Processing: {file_path}')
    
    # Load audio
    y, sr = load_audio(file_path)
    
    # Extract features
    features = extract_features(y, sr)
    
    # Visualize
    plot_waveform(y, sr)
    plot_chromagram(features['chroma'], sr)
    plot_onsets(y, sr, features['onsets'][0], features['onsets'][1])
    
    # Save features if output_dir is provided
    if output_dir:
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        output_path = os.path.join(output_dir, f'{file_name}_features.npz')
        np.savez(output_path, 
                 pitches=features['pitches'][0],
                 pitch_magnitudes=features['pitches'][1],
                 onsets=features['onsets'][0],
                 onset_env=features['onsets'][1],
                 chroma=features['chroma'])
        print(f'Features saved to: {output_path}')
    
    return features

# Example usage:
# file_path = os.path.join(DATA_DIR, 'example_guitar.wav')
# features = process_audio_file(file_path, OUTPUT_DIR)

## Next Steps

Based on these extracted features, we can move on to:

1. Note event detection
2. Pitch classification
3. String and fret assignment
4. Tab notation generation

These will be explored in subsequent notebooks.